In [4]:
import segmentation_models as sm
import numpy as np
import sys, os
from tensorflow.keras.utils import normalize
from tensorflow.keras.optimizers import Adam

In [5]:
sys.path.insert(1, 'E:/Research/course research projects/data-driven software engineering/gui-element-detection/src')
from constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
from constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path
from util.data_utils import LazyDataLoader, Splits
# from ..util.data_utils import LazyDataLoader, Splits
# from ..constants import VINS_MERGED_YOLO_SPLITS_IMAGES as imgs_path
# from ..constants import VINS_MERGED_YOLO_SPLITS_LABELS as labels_path

In [6]:
batch_size = 16
classes = 12 + 1
activation = 'softmax'
learning_rate = 0.0001
opt = Adam(learning_rate)
dice_loss = sm.losses.DiceLoss(
    class_weights=np.array([0.25, 0.25, 0.25, 0.25]))  # makes sense as the end result is highly imbalanced
focal_loss = sm.losses.CategoricalFocalLoss()  # why not Categorical CE ?
total_loss = dice_loss + (focal_loss * 1)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.IOUScore(threshold=0.9), sm.metrics.FScore(threshold=0.5),
           sm.metrics.Precision(), sm.metrics.Recall()]  # this is all they have :|

In [7]:
def get_data_loader(split):
    return LazyDataLoader(split=split, images_path=imgs_path, labels_path=labels_path, batch_size=batch_size,
                          n_classes=classes, resize=True, shape=(416, 416), normalize=True)

In [8]:
train_loader = get_data_loader(Splits.TRAIN)
validation_loader = get_data_loader(Splits.VALIDATION)
test_loader = get_data_loader(Splits.TEST)

In [9]:
RESNET_BACKBONE = 'resnet34'
preprocess_inputs = sm.get_preprocessing(RESNET_BACKBONE)

train_loader = preprocess_inputs(train_loader)
test_loader = preprocess_inputs(test_loader)
validation_loader = preprocess_inputs(validation_loader)

In [10]:
model1 = sm.Unet(RESNET_BACKBONE, encoder_weights='imagenet', classes=13, activation=activation)
model1.compile(opt, total_loss, metrics=metrics)

In [11]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['bn_data[0][0]']                
                                3)                                                          

In [12]:
history1 = model1.fit(train_loader,
                      epochs=100,
                      verbose=2,
                      validation_data=validation_loader)

TypeError: LazyDataLoader.__getitem__() takes 1 positional argument but 2 were given